In [1]:
from TSCFLP_instance import TSCFLPinstance
from TSCFLP_greedy_decoder import greedy_decoder
import torch
import torch.nn as nn
import numpy as np

In [2]:
class Generator(nn.Module):
    def __init__(self,I,J,population):
        super().__init__()
        self.population = population
        self.rede = nn.Sequential(
            nn.Linear(I+J,200),
            nn.ReLU(),
            nn.Linear(200,300),
            nn.ReLU(),
            nn.Linear(300,300),
            nn.ReLU(),
            nn.Linear(300,(I+J)*population),
            nn.Sigmoid()
        )
    def forward(self,ruido):
        x = self.rede(ruido)
        x = torch.reshape(x,(self.population,150))
        return x
        
        

In [3]:
class evaluate_population(nn.Module):
    def __init__(self,decoder):
        super(evaluate_population,self).__init__()
        self.is_inicialized = False 
        self.decoder = decoder
        self.best_fit = 0
        self.mean_current_pop= 0
        self.epochs_best_fit = []
        self.epochs_mean_fit = []
        
        
        
    def _loss(self,melhor,media):
        x = (melhor-media)/media
        loss = np.exp(-1*x)
        return loss
        
        

        
    
    def forward(self,population):
        pop_individual_fit = np.array([],dtype=int)
        for index,chromosome in enumerate(population):
            fit = self.decoder.decode(chromosome,True)
            pop_individual_fit = np.append(pop_individual_fit,fit)
        
        if self.is_inicialized == False:
            self.best_fit = np.min(pop_individual_fit)
            self.mean_current_pop = np.mean(pop_individual_fit)
                
            self.epochs_best_fit.append(self.best_fit)
            self.epochs_mean_fit.append(self.mean_current_pop)
            
    
            loss = self._loss(self.best_fit,self.mean_current_pop)
            
            
    
            self.is_inicialized = True
        else:
            self.mean_current_pop = np.mean(pop_individual_fit)
     
            
            loss = self._loss(self.best_fit,self.mean_current_pop)
            
            best_individual_fit = np.min(pop_individual_fit)
                          
            if best_individual_fit < self.best_fit:
                self.best_fit = best_individual_fit
                
                 
        return torch.tensor([loss])

In [4]:
model = Generator(50,100,200)
instancia = TSCFLPinstance(r"C:\Users\Felipe\Projetos\TSCFLP\Instances\PSC1-C1-50.txt")
decoder = greedy_decoder(instancia)
evaluator = evaluate_population(decoder)
lr = 0.01
optimizer = torch.optim.SGD(model.parameters(),lr=lr)
Epochs = 300

In [5]:
def train_loop(model,evaluator,optimizer,Epochs):
    epochs_loss = []
    
    
    for epoch in range(Epochs):
        ruido = torch.randn(150)
        print(ruido)
        population = model(ruido)
        
  
        optimizer.zero_grad()
        loss = evaluator(population)
        loss.requires_grad_()
        
        loss.backward()
        
        optimizer.step()
        epochs_loss.append(loss.item())
        
        if epoch%10==0:
            print(f'Melhor individuo:{evaluator.best_fit} Média da população: {evaluator.mean_current_pop}')
    return epochs_loss,evaluator

In [6]:
#loss,evaluete =  train_loop(model,evaluator,optimizer,Epochs)